In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip3 install tensorflow_decision_forests

In [ ]:
import tensorflow_decision_forests as tfdf
import tensorflow as tf
from tensorflow import keras

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/test.csv')
submission = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/sample_submission.csv')

In [ ]:
X = train.drop('id', axis = 1)
label = 'loss'

In [ ]:
test_X= test.drop('id',axis = 1)

In [ ]:
# pre defined hyper parameters tunnings
tfdf.keras.GradientBoostedTreesModel.predefined_hyperparameters()

In [ ]:
# Split the dataset into a training and a testing dataset.

def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


train_ds_pd, test_ds_pd = split_dataset(X)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

In [ ]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)
model_1 = tfdf.keras.GradientBoostedTreesModel(hyperparameter_template="benchmark_rank1", task=tfdf.keras.Task.REGRESSION)
model_1.compile(tf.keras.metrics.RootMeanSquaredError())
model_1.fit(train_ds)

In [ ]:
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)

In [ ]:
# Evaluate the model on the test dataset.
evaluation = model_1.evaluate(test_ds)

print(evaluation)


In [ ]:
# converting test dataframe to keras dataset
test = tfdf.keras.pd_dataframe_to_tf_dataset(test_X, task=tfdf.keras.Task.REGRESSION)

In [ ]:
preds = model_1.predict(test)

In [ ]:
submission['loss'] = preds

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission_xgboost_tf.csv', index=False)